# [PageRank](https://zh.wikipedia.org/wiki/PageRank)

PageRank，網頁排名，又稱網頁級別、Google左側排名或佩奇排名，是一種由搜尋引擎根據 **網頁之間相互的超連結計算** 的技術，而作為網頁排名的要素之一，以Google公司創辦人拉里·佩奇（Larry Page）之姓來命名。

這個方程式引入了 **隨機瀏覽** 的概念，即有人上網無聊隨機開啟一些頁面，點一些連結。一個頁面的PageRank值也影響了它被隨機瀏覽的機率。為了便於理解，這裡假設 **上網者不斷點網頁上的連結，最終到了一個沒有任何鏈出頁面的網頁，這時候上網者會隨機到另外的網頁開始瀏覽**。

- - -

假設符合搜尋條件的只有這三個網頁, A, B and C

* A 網頁有兩個連結一個連到 B 一個連到 C, 也就是有 0.5 的機率連到 B, 有 0.5 的機率連到 C
* B 網頁有一個連結連到 C, 也就是有 1.0 的機率連到 C
* C 網頁有一個連結連到 A, 也就是有 1.0 的機率連到 A

![Certificate](http://pr.efactory.de/i/bsp1.gif)

假設一個瀏覽者在 A 網頁, 他開始 **隨機瀏覽**, 因為假設是 **上網者不斷點網頁上的連結**, 所以, 下一次會瀏覽 B 的機率是 0.5, 會瀏覽 C 的機率是 0.5

再下一次會瀏覽 C 的機率是 0.5, 會瀏覽 A 的機率是 0.5

其實畫個樹狀圖很容易觀察上網者現在所在頁面的機率, 我懶的畫

這其實在統計上是典型的 [**馬可夫鍊**](https://zh.wikipedia.org/wiki/%E9%A9%AC%E5%B0%94%E5%8F%AF%E5%A4%AB%E9%93%BE) 問題

In [5]:
import numpy as np
T = np.array([[0,0,1], [0.5,0,0], [0.5,1,0]])
T

array([[ 0. ,  0. ,  1. ],
       [ 0.5,  0. ,  0. ],
       [ 0.5,  1. ,  0. ]])

`T` 一般稱作 **轉移矩陣**, `T[1,0] = 0.5` 表示瀏覽者從 A 連到 B 的機率, `T[0,0] = 0.0` 表示瀏覽者從 A 連到 A 的機率, 以此類推

In [52]:
p = np.array([1, 0, 0])
p

array([1, 0, 0])

`p` 一般稱作 **狀態矩陣**, 因為我前面是舉例瀏覽者在 A 網頁, 所以狀態矩陣是 `array([1, 0, 0])`, 如果一開始在 B, 狀態矩陣是 `array([0, 1, 0])`, 以此類推

通常是設定成 `np.array([1/3, 1/3, 1/3])`, 不過, 這不是很重要, 可以忽略這句話

所以, 我如果想知道瀏覽者下一次停留頁面的機率, 我只要用下面公式

In [14]:
T.dot(p)

array([ 0. ,  0.5,  0.5])

表示下一次會瀏覽 B 的機率是 0.5, 會瀏覽 C 的機率是 0.5

再下一次呢?

In [47]:
T.dot(T).dot(p)

array([ 0.5,  0. ,  0.5])

表示下一次會瀏覽 C 的機率是 0.5, 會瀏覽 A 的機率是 0.5

其實計算方式是有規律的(`T.dot(T).dot(T)....dot(T).dot(p)`), 寫個 `function`, `iter_n` 表示經過幾次 **隨機瀏覽**, 回傳所在頁面機率

In [48]:
def get_p(iter_n):
    return np.linalg.matrix_power(T, iter_n).dot(p)

所以, 假設一直瀏覽下去, 會怎樣呢?

In [53]:
for i in range(50):
    print i, get_p(i)

0 [ 1.  0.  0.]
1 [ 0.   0.5  0.5]
2 [ 0.5  0.   0.5]
3 [ 0.5   0.25  0.25]
4 [ 0.25  0.25  0.5 ]
5 [ 0.5    0.125  0.375]
6 [ 0.375  0.25   0.375]
7 [ 0.375   0.1875  0.4375]
8 [ 0.4375  0.1875  0.375 ]
9 [ 0.375    0.21875  0.40625]
10 [ 0.40625  0.1875   0.40625]
11 [ 0.40625   0.203125  0.390625]
12 [ 0.390625  0.203125  0.40625 ]
13 [ 0.40625    0.1953125  0.3984375]
14 [ 0.3984375  0.203125   0.3984375]
15 [ 0.3984375   0.19921875  0.40234375]
16 [ 0.40234375  0.19921875  0.3984375 ]
17 [ 0.3984375   0.20117188  0.40039062]
18 [ 0.40039062  0.19921875  0.40039062]
19 [ 0.40039062  0.20019531  0.39941406]
20 [ 0.39941406  0.20019531  0.40039062]
21 [ 0.40039062  0.19970703  0.39990234]
22 [ 0.39990234  0.20019531  0.39990234]
23 [ 0.39990234  0.19995117  0.40014648]
24 [ 0.40014648  0.19995117  0.39990234]
25 [ 0.39990234  0.20007324  0.40002441]
26 [ 0.40002441  0.19995117  0.40002441]
27 [ 0.40002441  0.20001221  0.39996338]
28 [ 0.39996338  0.20001221  0.40002441]
29 [ 0.400024

假設瀏覽動作重複 50 次, 在 A, B and C的機率會慢慢 **收斂** 至 0.4, 0.2, 0.4

這裡就姑且當作 A 的 pagerank 是 0.4, B 的 pagerank 是 0.2, C 的 pagerank 是 0.4 (**其實不完全是這樣算的**)

如果妳都看不懂我在幹嘛, 重點就是

**你一直看一直看一直看一直看一直看一直看一直看一直看一直看一直看, 停在 A 跟 C 的機率最高**, 而這個機率是由 **網頁之間相互的超連結計算**

**你一直看一直看一直看一直看一直看一直看一直看一直看一直看一直看, 停在 A 跟 C 的機率最高**, 而這個機率是由 **網頁之間相互的超連結計算**

**你一直看一直看一直看一直看一直看一直看一直看一直看一直看一直看, 停在 A 跟 C 的機率最高**, 而這個機率是由 **網頁之間相互的超連結計算**

所以, A and C 會顯示在最前面, B 最後面

- - -

這只是我舉的 **超簡單範例**, 真正是這樣算的

![Certificate](https://wikimedia.org/api/rest_v1/media/math/render/svg/71b912c44a05c1fea711d90ca5ee81c2529b3e6f)

我去掉很多複雜的運算(**其實是我看不懂的地方**), 但 pagerank 就是用這些觀念堆積而成

* `d` 是一個很有趣的數字, 它可以用來解決當一個沒有向外連接的頁面, 設定 0.85 也是有原因的, 詳情請見 [PageRank 完整版本](https://zh.wikipedia.org/wiki/PageRank#.E5.AE.8C.E6.95.B4.E7.89.88.E6.9C.AC)

* pagerank 對新頁面是不公平的, 因為新頁面並不會有很多外連

* 為什麼維基百科總在第一名? 進得去出不來????????????

* [Google：別再用 PageRank 當評量標準，我們不打算更新了](http://technews.tw/2014/11/25/google-stop-using-pagerank-as-a-metric-we-wont-be-updating-it-in-the-future/)

* 雖然感覺 pagerank 不潮了, 但有人把它拿來做在文字上, 還能找 **關鍵字**, 神奇吧

- - -

講這麼多, 不介紹個工具, 也沒屁用

我找到的是 [networkx](http://networkx.readthedocs.io/en/networkx-1.10/) 這個 library

In [1]:
import networkx as nx  

以下程式可以重現我上面舉的範例

In [5]:
D = nx.DiGraph()
D.add_weighted_edges_from([('A', 'B', 0.5), ('A', 'C', 0.5), ('B', 'C', 1), ('C', 'A', 1)])
print nx.pagerank(D, alpha=0.85, personalization={'A': 0.33, 'B': 0.33, 'C': 0.33}, max_iter=50)

{'A': 0.3877894427072592, 'C': 0.3974000441421557, 'B': 0.21481051315058516}
